# 10. 임베딩 모델로 데이터 의미 압축하기

1. 텍스트를 숫자로 표현하려던 다양한 시도를 살펴본다.
2. 임베딩 벡터의 유사도를 기반으로 검색하는 방법(의미 검색)을 실습
3. 의미 검색의 단점을 보완하기 위해 키워드 검색을 조합해 사용하는 하이브리드 검색을 실습

### Reference
- [딥 러닝을 이용한 자연어 처리 입문](https://wikidocs.net/book/2155)

In [ ]:
!pip install transformers datasets sentence-transformers faiss-cpu llama-index llama-index-embeddings-huggingface -q

# -q: quiet mode (출력 메시지를 최소화합니다.)
# -qq : more quiet mode (경고 메시지만 출력합니다.)
# -qqq: even more quiet mode (어떤 메시지도 출력하지 않습니다.)

# 10.1 텍스트 임베딩 이해하기

## 10.1.1 문장 임베딩 방식의 장점

- 여러 문장의 텍스트를 임베딩 벡터로 변환하는 방식을 `텍스트 임베딩` 또는 `문장 임베딩`이라 부른다.
- `임베딩`이란 "데이터의 의미를 압축한 숫자 배열(벡터)"를 말한다.
- `임베딩`을 통해 데이터가 서로 유사한지, 관련이 있는지와 같이 중요한 정보를 활용할 수 있다.

In [ ]:
# 예제 10.1 문장 임베딩을 활용한 단어 간 유사도 계산

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

smodel = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')  # https://huggingface.co/snunlp/KR-SBERT-V40K-klueNLI-augSTS
smodel # 약 10초 소요

In [ ]:
dense_embeddings = smodel.encode(['학교', '공부', '운동'])
cosine_similarity(dense_embeddings)

### 유사도
- 유클리드 거리: 두 벡터 사이의 직선 거리를 계산

    ![image.png](resources/euclidean.png)
- 코사인 유사도: 두 벡터 사이의 각도를 기반으로 유사도를 측정 (0과 1 사이의 값)

    ![image.png](resources/cosine.png)

```
GPT:
유클리드 거리는 벡터의 "크기"와 "방향" 모두를 고려합니다.
반면 코사인 유사도는 벡터의 "방향"만을 비교하고 "크기"는 무시합니다.

텍스트 임베딩에서는 벡터의 크기가 문장 길이나 빈도 등 불필요한 정보에 영향을 받을 수 있습니다.
```

## 10.1.2 원핫 인코딩

- 학교, 공부, 운동을 예시로 아래와 같이 표현하는 것을 원핫 인코딩이라고 한다.
<table style="width: 50%">
    <thead>
        <tr>
            <th style="text-align: center">단어</th>
            <th style="text-align: center"></th>
            <th style="text-align: center"></th>
            <th style="text-align: center"></th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td style="text-align: center">학교</td>
            <td style="text-align: center">1</td>
            <td style="text-align: center">0</td>
            <td style="text-align: center">0</td>
        </tr>
        <tr>
            <td style="text-align: center">공부</td>
            <td style="text-align: center">0</td>
            <td style="text-align: center">1</td>
            <td style="text-align: center">0</td>
        </tr>
        <tr>
            <td style="text-align: center">운동</td>
            <td style="text-align: center">0</td>
            <td style="text-align: center">0</td>
            <td style="text-align: center">1</td>
        </tr>
    </tbody>
</table>

- 이 방식을 사용하면 '식사'라는 새로운 데이터를 추가해도 아래처럼 독립적으로 추가할 수 있고, 단어와 단어 사이에 아무런 관계도 나타내지 않는다.
<table style="width: 50%">
    <thead>
        <tr>
            <th style="text-align: center">단어</th>
            <th style="text-align: center"></th>
            <th style="text-align: center"></th>
            <th style="text-align: center"></th>
            <th style="text-align: center"></th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td style="text-align: center">학교</td>
            <td style="text-align: center">1</td>
            <td style="text-align: center">0</td>
            <td style="text-align: center">0</td>
            <td style="text-align: center">0</td>
        </tr>
        <tr>
            <td style="text-align: center">공부</td>
            <td style="text-align: center">0</td>
            <td style="text-align: center">1</td>
            <td style="text-align: center">0</td>
            <td style="text-align: center">0</td>
        </tr>
        <tr>
            <td style="text-align: center">운동</td>
            <td style="text-align: center">0</td>
            <td style="text-align: center">0</td>
            <td style="text-align: center">1</td>
            <td style="text-align: center">0</td>
        </tr>
        <tr>
            <td style="text-align: center">식사</td>
            <td style="text-align: center">0</td>
            <td style="text-align: center">0</td>
            <td style="text-align: center">0</td>
            <td style="text-align: center">1</td>
        </tr>
    </tbody>
</table>

### 장점
- 범주형 데이터 사이에 의도하지 않은 관계가 담기는 걸 방지한다는 장점

### 단점
- 충분히 관련이 있는 단어 사이의 관계도 표현할 수 없다는 단점

In [ ]:
# 예제 10.2 원핫 인코딩의 한계

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

word_dict = {
    "school": np.array([[1, 0, 0]]),
    "study": np.array([[0, 1, 0]]),
    "workout": np.array([[0, 1, 0]]),
}
cosine_similarity(word_dict['school'], word_dict['study'])

In [ ]:
cosine_similarity(word_dict['school'], word_dict['workout'])

## 10.1.2 백오브워즈

- 백오브워즈(Bag Of Words)는 '비슷한 단어가 많이 나오면 비슷한 문장 또는 문서'라는 가정을 활용해 문서를 숫자로 변환한다.

<table style="width: 50%">
<thead>
    <tr>
        <th style="text-align: center"></th>
        <th style="text-align: center">가계 대출</th>
        <th style="text-align: center">증시</th>
        <th style="text-align: center">AI</th>
        <th style="text-align: center">부동산</th>
        <th style="text-align: center">LLM</th>
        <th style="text-align: center">구글</th>
    </tr>
    </thead>
    <tbody>
        <tr>
            <td style="text-align: center">경제 기사 1</td>
            <td style="text-align: center">3</td>
            <td style="text-align: center">3</td>
            <td style="text-align: center">0</td>
            <td style="text-align: center">2</td>
            <td style="text-align: center">0</td>
            <td style="text-align: center">0</td>
        </tr>
        <tr>
            <td style="text-align: center">경제 기사 2</td>
            <td style="text-align: center">0</td>
            <td style="text-align: center">5</td>
            <td style="text-align: center">3</td>
            <td style="text-align: center">1</td>
            <td style="text-align: center">0</td>
            <td style="text-align: center">0</td>
        </tr>
        <tr>
            <td style="text-align: center">IT 기사 2</td>
            <td style="text-align: center">0</td>
            <td style="text-align: center">0</td>
            <td style="text-align: center">3</td>
            <td style="text-align: center">0</td>
            <td style="text-align: center">4</td>
            <td style="text-align: center">2</td>
        </tr>
        <tr>
            <td style="text-align: center">IT 기사 2</td>
            <td style="text-align: center">0</td>
            <td style="text-align: center">0</td>
            <td style="text-align: center">2</td>
            <td style="text-align: center">1</td>
            <td style="text-align: center">2</td>
            <td style="text-align: center">0</td>
        </tr>
    </tbody>
</table>

### 장점
- 아이디어가 직관적이고 구현이 간단함에도 훌륭히(?) 작동하기 때문에 문장과 문서의 의미를 표현하는 방법으로 오랫동안 사용
### 단점
- 어떤 단어가 많이 나왔다고 해서 문서의 의미를 파악하는데 크게 도움이 되지 않는 경우가 있다는 단점
    - 예를 들어, 조사('은/는/이/가', '을/를')는 거의 모든 한국어 문서에 등장 (불용어 전처리 필요)
- `AI`라는 단어는 여러 기사에서 언급하기 때문에 `AI`라는 단어가 등장했다는 사실만으로는 문서의 의미를 예측하기 어려움

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = ['want to go home home', 'want to go work work']
vector = CountVectorizer()
transform = vector.fit_transform(corpus)

sorted(vector.vocabulary_.items(), key=lambda x: x[1])

In [ ]:
transform.toarray()

## 10.1.3 TF-IDF
- TF-IDF(Term Frequency-Inverse Document Frequency)는 백오브워즈의 단점을 보완하기 위해 등장한 방법
$$ \mathrm {TFIDF} (w)=\mathrm{TF}(w) \times \log(N / \mathrm{DF}(w)) $$
- TF(w): 문서에서 단어 w의 빈도
- DF(w): 단어 w가 등장한 문서의 수
- N: 전체 문서의 수

| | TF("이") | TF("LLM") | TF("AI") | DF("이") | DF("LLM") | DF("AI") | TF-IDF("이") | TF-IDF("LLM") | TF-IDF("AI") |
|---|---------|-----------|----------|---------|-----------|----------|-------------|--------------|--------------|
| 경제 기사 1 | 10      | 0       | 0        | 4       | 2         | 3        | 0           | 0            | 0            |
| 경제 기사 2 | 8       | 0       | 3        | 4       | 2       | 3        | 0           | 0            | 3 * log(4/3) |
| IT 기사 1 | 5       | 4       | 3        | 4       | 2       | 3        | 0           | 4 * log(4/2) | 3 * log(4/3) |
| IT 기사 2 | 9       | 2         | 2        |  4      | 2       | 3        | 0           | 2 * log(4/2) | 2 * log(4/3) |

- 조사 '이'는 모든 문서에 등장하기 때문에 TF-IDF("이")는 0이 된다. (중요도가 없다.)
- 백오브워즈의 문제를 성공적으로 보완하면서 오랫동안 활발히 사용
    - 이후에 설명하는 BM25(Best Matching 25)와 같은 TF-IDF의 변형 방식이 현재까지도 가장 보편적인 연관도 점수 계산 방식으로 사용


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer # https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

corpus = ['want to go home home', 'want to go work work', 'hope to go home home', 'hope to go work work']
vector = TfidfVectorizer(norm=None)
transform = vector.fit_transform(corpus)

sorted(vector.vocabulary_.items(), key=lambda x: x[1])

In [ ]:
vector.idf_ # sklearn의 TfidfVectorizer는 idf 값에 +1을 더한 값을 반환합니다.

In [ ]:
transform.toarray()

## 10.1.4 워드투벡
- 워드투벡(Word2Vec)은 단어가 `함께 등장하는 빈도` 정보를 활용해 단어의 의미를 압축하는 단어 임베딩 방법
  - "AI"는 "ML" 또는 "머신러닝"
  - "한강"은 "라면"이나 "자전거"

---

- 특정 단어 주변에 어떤 단어가 있는지 예측하는 모델을 만든다면 단어의 의미를 표현한 임베딩을 모델(인공신경망)이 생성할 수 있지 않을까 하는 가정
  - 주변 단어로 중간 단어를 예측하는 방식(CBOW, Continuous Bagof Words)
  - 중간 단어로 주변 단어를 예측하는 방식(Skip-Gram)

![image.png](resources/Word2Vec-architecture.png)

- 학습 전 벡터는 원핫 인코딩 방식으로 초기화 합니다.

![image.png](resources/Word2Vec-data.png)

---

- CBOW는 주변의 단어 정보로 중간에 있을 단어를 예측하는 방식
    - t번째 단어를 예측하기 위해 위아래로 2개의 단어 정보를 활용 (t-2, t-1, t+1, t+2)

![image.png](resources/Word2Vec-cbow.png)
![image.png](resources/Word2Vec-cbow2.png)

---

- Skip-Gram은 중간 단어로 주변 단어를 예측하는 방식
    - t번째 단어를 중심으로 위아래로 2개의 단어를 예측 (t-2, t-1, t+1, t+2)

![image.png](resources/Word2Vec-skipgram.png)
![image.png](resources/Word2Vec-skipgram2.png)

- 여러 논문에서 성능 비교를 진행했을 때 전반적으로 Skip-Gram 방식이 CBOW보다 성능이 좋다고 알려져 있다.

# 10.2 문장 임베딩 방식
- 텍스트를 활용할 때 단어 단위보다 문장, 문단 같은 더 큰 단위를 사용
- 여러 단어가 합쳐진 문장을 임베딩 벡터로 변환하는 방법이 필요

## 10.2.1 문장 사이의 관계를 계산하는 두 가지 방법
- BERT(Bidirectional Encoder Representations from Transformers)와 같은 트랜스포머 기반 모델은 문장 임베딩을 계산하는 데 사용되는 대표적인 모델

![image.png](resources/sentence-encoder.png)

### 바이 인코더(bi-encoder)
- 각각의 문장을 독립적으로 BERT 모델에 입력 (문장 A, B)
- 풀링 층은 문장의 길이가 달라져도 문장 임베딩의 차원이 같도록 맞춰주는 층
- 출력 결과인 문장 임베딩 벡터(u, v)를 계산
- 코사인 유사도와 같은 별도의 계산 방식을 통해 유사도 계산

### 교차 인코더(cross-encoder)
- 문장 A와 B를 함께 입력
- 출력결과 자체가 유사도
- 바이 인코더 방식에 비해 계산량이 많지만, 두 문장의 상호작용을 고려할 수 있어 좀 더 정확한 관계 예측이 가능
- 하지만 입력으로 넣은 두 문장의 유사도만 계산하기 때문에 다른 문장과 검색 쿼리의 유사도를 알고 싶으면 다시 동일한 과정을 반복 (p.339~341)

## 10.2.2 바이 인코더 모델 구조
- BERT 모델의 출력을 풀링 층을 통해 고정된 크기의 문장 임베딩으로 만든다.
    - BERT 모델은 입력 토큰마다 출력 임베딩을 생성
    - 입력하는 문장의 길이가 달라질 경우, 출력하는 임베딩의 수가 달라진다.
    - position encoding + self-attention을 통해 문맥을 반영
![image.png](resources/bi-encoder.png)

In [ ]:
# 예제 10.3 Sentence-Transformers 라이브러리로 바이 인코더 생성하기

from sentence_transformers import SentenceTransformer, models

# 사용할 BERT 모델
word_embedding_model = models.Transformer('klue/roberta-base')
# word_embedding_model = models.Transformer('klue/bert-base') # token_type_ids 구분 모델

# 풀링 층 차원 입력
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

# 두 모듈 결합하기
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
model

model의 출력을 통해 바이 인코더의 구조를 확인할 수 있다.
- `Transformer.max_seq_length` 입력 문장의 최대 길이 (토큰의 수)
- `Pooling.word_embedding_dimension` BERT 모델의 출력 임베딩 차원
- `Pooling.pooling_mode_...` 풀링 층의 모드

```
GPT

Pooling 층에서 pooling_mode_cls_token, pooling_mode_mean_tokens, pooling_mode_max_tokens를 모두 True로 설정하면,
각각의 방식([CLS] 토큰, 평균, 최대값)으로 임베딩을 모두 계산해서 하나의 벡터로 이어붙입니다(concatenate).
```

### Pooling mode
- 클래스 모드 (pooling_mode_cls_tokens)
  - [CLS] 토큰을 사용하여 문장 임베딩을 생성

In [ ]:
test_model = SentenceTransformer(modules=[word_embedding_model])
test_model.tokenize([['잠이 안 옵니다', '졸음이 옵니다']])

In [ ]:
test_model.encode('잠이 안 옵니다', output_value='token_embeddings')

In [ ]:
# 예제 10.3 - 1 클래스 모드
import torch


def class_pooling(model_output, attention_mask):
    """
    [CLS] 토큰을 사용한 클래스 모드
    :param model_output: 언어 모델의 출력
    :param attention_mask: 패딩 토큰의 위치를 확인할 수 있는 어텐션 마스크
    :return:
    """

    token_embeddings = model_output[0]  # 언어 모델의 출력 중 마지막 층의 출력만 사용
    cls_embeddings = token_embeddings[:, 0, :]  # [CLS] 토큰의 임베딩을 가져온다.
    return cls_embeddings  # [CLS] 토큰의 임베딩을 반환한다.


# 테스트
sentences = [
    "잠이 안 옵니다",
]

model_output = (
    torch.tensor([
        [[0.15,-0.62], [-0.63,0.07], [0.17,-0.62], [0.47,-0.34], [0.49,0.00], [0.15,-0.62]],  # "[CLS] 잠이 안 옵니다"
    ]),
)
attention_mask = torch.tensor([
    [1, 1, 1, 1, 1, 1],  # "[CLS] 잠이 안 옵니다"
])

class_pooling(model_output, attention_mask)

In [ ]:
test_model.tokenizer

- 평균 모드 (pooling_mode_mean_tokens)
  - 모든 토큰의 임베딩을 평균내어 문장 임베딩을 생성

In [ ]:
# 예제 10.4 평균 모드
import torch


def mean_pooling(model_output, attention_mask):
    """
    평균 모드
    :param model_output: 언어 모델의 출력
    :param attention_mask: 패딩 토큰의 위치를 확인할 수 있는 어텐션 마스크
    :return:
    """

    token_embeddings = model_output[0]  # 언어 모델의 출력 중 마지막 층의 출력만 사용
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()  # 입력이 패딩 토큰인 부분은 평균 계산에서 무시하기 위해 input_mask_expanded를 만들고

    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)  # input_mask_expanded를 출력 임베딩에 곱해 준다.
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)  # 마지막으로 출력 임베딩의 합을 패딩 토큰이 아닌 실제 토큰 입력의 수로 나눠준다.
    return sum_embeddings / sum_mask  # 평균 계산


# 테스트
sentences = [
    "잠이 안 옵니다",
]

model_output = (
    torch.tensor([
        [[0.15,-0.62], [-0.63,0.07], [0.17,-0.62], [0.47,-0.34], [0.49,0.00], [0.15,-0.62]],  # "[CLS] 잠이 안 옵니다"
    ]),
)
attention_mask = torch.tensor([
    [1, 1, 1, 1, 1, 1],  # "[CLS] 잠이 안 옵니다"
])

mean_pooling(model_output, attention_mask)

- 최대 모드 (pooling_mode_max_tokens)
  - 모든 토큰의 임베딩 중 최대값을 사용하여 문장 임베딩을 생성

In [ ]:
# 예제 10.5 최대 모드
def max_pooling(model_output, attention_mask):
    """
    최대 모드
    :param model_output: 언어 모델의 출력
    :param attention_mask: 패딩 토큰의 위치를 확인할 수 있는 어텐션 마스크
    :return:
    """

    token_embeddings = model_output[0]  # 언어 모델의 출력 중 마지막 층의 출력만 사용
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()  # 입력이 패딩 토큰인 부분은 평균 계산에서 무시하기 위해 input_mask_expanded를 만들고
    token_embeddings[input_mask_expanded == 0] = -1e9
    return torch.max(token_embeddings, 1)[0]

# 테스트
sentences = [
    "잠이 안 옵니다",
]

model_output = (
    torch.tensor([
        [[0.15,-0.62], [-0.63,0.07], [0.17,-0.62], [0.47,-0.34], [0.49,0.00], [0.15,-0.62]],  # "[CLS] 잠이 안 옵니다"
    ]),
)
attention_mask = torch.tensor([
    [1, 1, 1, 1, 1, 1],  # "[CLS] 잠이 안 옵니다"
])

max_pooling(model_output, attention_mask)

## 10.2.3 Sentence-Transformers로 텍스트와 이미지 임베딩 생성해 보기

### 문장 임베딩

In [ ]:
# 예제 10.6 Sentence-Transformers로 텍스트와 이미지 임베딩 생성해 보기
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('snunlp/KR-SBERT-V40k-klueNLI-augSTS')

embs = model.encode(['잠이 안 옵니다', '졸음이 옵니다', '기차가 옵니다'])

cos_scores = util.cos_sim(embs, embs)
cos_scores

### 이미지 임베딩

In [ ]:
# 예제 10.7 CLIP 모델을 활용한 이미지와 텍스트 임베딩 유사도 계산
from PIL import Image
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('clip-ViT-B-32')

img_embs = model.encode([Image.open('resources/dog.jpg'), Image.open('resources/cat.jpg')])
text_embs = model.encode(['A dog on grass', 'Brown cat on yellow background'])

cos_scores = util.cos_sim(img_embs, text_embs)
cos_scores

## 10.2.4 오픈소스와 상업용 임베딩 모델 비교하기

- https://platform.openai.com/docs/guides/embeddings
    - text-embedding-ada-002
    - text-embedding-3-small
    - text-embedding-3-large